In [3]:
from dimensionality_reduction import dr_action
from classification import cla_action
from visualization import vis_action
from preprocessing import pp_action
from clustering import clu_action
from keywords import kw_action
import json


Load the configuration file in a dictionary. It contains the paths to the data folders.

In [2]:
config = json.load(open('config.json', 'r'))

### Data selection

Select a subset of the dataset. Then the selected subset will be split into a training and a testing set.


In [4]:
samples_data = pp_action.pre_process(config)
pp_action.split_show_data(samples_data)

Please choose the subset of data to workon on:
l for all labeled samples
k for samples of families mydoom, gepys, lamer, neshta, bladabindi, flystudio, eorezo
s for 8 samples of families mydoom, gepys, bladabindi, flystudio
f for a single family
b for a balanced subset of samples
q to quit
s

6 train samples belonging to 4 malware families
Malware family:      flystudio       Number of samples:   2   
Malware family:        mydoom        Number of samples:   2   
Malware family:        gepys         Number of samples:   1   
Malware family:      bladabindi      Number of samples:   1   

2 test samples belonging to 2 malware families
Malware family:        gepys         Number of samples:   1   
Malware family:      bladabindi      Number of samples:   1   


         family  fam_num  selected  train  test
count     58180  58180.0       8.0    6.0   2.0
unique     1270   1270.0       NaN    NaN   NaN
top     allaple     42.0       NaN    NaN   NaN
freq      10569  10569.0       NaN    

In [5]:
reduced,inverse_reduced = dr_action.dimensionality_reduction(samples_data, config)


Please select a dimensionality reduction technique:
pca
svd
tsne
lda
s to skip dimensionality reduction
q to quit
pca
Please select the desired number of components (q to quit)
8
Performing dimensionality reduction using PCA
Processing documents from 0 to 299
Loading Tf-Idf of 8 documents
4757760
4757760
4757760
4757760
(8, 297360)
Explained Variance Ratio
1.0
Transforming documents from 0 to 299
Loading Tf-Idf of 8 documents


/home/gio/.virtualenvs/malcluster/lib/python3.5/site-packages/sklearn/decomposition/incremental_pca.py:269: RuntimeWarning:

Mean of empty slice.

/home/gio/.virtualenvs/malcluster/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning:

invalid value encountered in double_scalars



4757760
4757760
4757760
4757760
(8, 297360)


In [6]:
reduced

array([[ -7.23904238e+02,  -4.57833979e+02,   5.45297850e+00,
          4.67942460e+02,   1.93507208e+01,  -3.96869094e+01,
         -1.02556990e+00,   1.30101941e-12],
       [ -7.34239659e+02,   2.41537866e+03,  -2.31140081e+00,
         -5.11048638e+00,  -5.94916436e-02,   7.01786899e-01,
          3.75944510e-02,   2.59937037e-13],
       [ -7.26193438e+02,  -4.90024865e+02,   4.21166540e+00,
         -1.72276623e+02,   2.96232788e+02,  -1.11664934e+02,
         -4.79851086e+00,  -5.24684029e-13],
       [  2.14477254e+03,  -9.18776777e+00,  -6.64297965e+02,
          2.77962707e-01,  -1.94530553e-02,  -6.94810748e-02,
          3.40729901e-02,   9.19035681e-13],
       [ -7.29623001e+02,  -4.92646185e+02,   3.66215611e+00,
         -7.39607042e+01,  -1.58092353e+01,   1.37657529e+02,
          8.70767786e+01,   6.04517304e-13],
       [ -7.28718888e+02,  -4.91780604e+02,   3.30562311e+00,
         -7.31166977e+01,  -2.83358503e+01,   1.62611011e+02,
         -7.92090770e+01,  -4.6

In [10]:
from pprint import  pprint

pprint(inverse_reduced[0])



array([  2.30082856e-13,  -7.67147209e-13,  -2.92210700e-13, ...,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00])
